In [1]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster

import requests
import json
from utils_data import get_contours_city_ign, add_area_gdf, get_data_carroyee_city 
from utils_data import get_euclidian_adjacency_from_df, get_adjacency_from_gdf

In [3]:
file_ign = '../Data/BDCARTO/44_Loire_Atlantique/data.gpkg'
file_carroye = '../Data/Filosofi2017_carreaux_200m_met.gpkg'
code_insee=44055
crs_meters = 'EPSG:2154'
crs_angles = 'EPSG:4326'
waiting_time_api=0.1

processed_dir = '../data/processed'

export_file_df_adjacency=f"{processed_dir}/gdf_city_200_with_dist_{code_insee}.csv"
export_file_np_adjacency=f"{processed_dir}/adjacency_matrix_200_{code_insee}.npy"

In [4]:
gdf_contours_city, geometry_contours_city = get_contours_city_ign(file_ign, code_insee, crs=crs_angles)
center_lon, center_lat = geometry_contours_city.centroid.coords[0]

In [5]:
gdf_carroye_city = get_data_carroyee_city(file_carroye, geometry_contours_city, crs=crs_angles)

In [22]:
cols = ['Idcar_200m', 'I_est_200', 'Ind', 'geometry', 'Men_coll', 'Men_mais', 'Ind_snv']
gdf_carroye_city_kept = gdf_carroye_city[cols]
gdf_carroye_city_kept = gdf_carroye_city_kept.rename(columns={'Idcar_200m':'id', 'I_est_200':'estimated', 'Ind':'population', 'Ind_snv':'niveau_de_vie'}) 
gdf_carroye_city_kept = add_area_gdf(gdf_carroye_city_kept, crs_meters=crs_meters, crs_angles=crs_angles)

gdf_carroye_city_kept['density_km2'] = gdf_carroye_city_kept['population'] / gdf_carroye_city_kept['area_km2']
gdf_carroye_city_kept['ratio_appart'] = gdf_carroye_city_kept['Men_coll'] / (gdf_carroye_city_kept['Men_coll'] + gdf_carroye_city_kept['Men_mais'])
gdf_carroye_city_kept['niveau_de_vie_moyen'] = gdf_carroye_city_kept['niveau_de_vie'] / gdf_city_with_dist['population']


gdf_carroye_city_kept.drop(columns=['area_km2', 'Men_coll', 'Men_mais', 'niveau_de_vie'], inplace=True)

In [23]:
gdf_carroye_city_kept.head()

,id,estimated,population,geometry,density_km2,ratio_appart,niveau_de_vie_moyen
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,0.150000,29316.704762
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,0.000000,23425.138462
2,CRS3035RES200mN2761400E3392000,1,2.0,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,...",50.086451,0.000000,28876.200000
3,CRS3035RES200mN2761600E3391200,0,93.5,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009...",2341.540973,0.542373,24798.703743
4,CRS3035RES200mN2761600E3391400,0,72.0,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039...",1803.111518,0.055556,27235.497222


In [25]:
# gdf_city_200_with_dist, adjacency_matrix = get_adjacency_from_gdf(gdf_carroye_city_kept, delay=waiting_time_api, crs_angles=crs_angles, crs_meters=crs_meters)
gdf_city_200_with_dist, adjacency_matrix = get_euclidian_adjacency_from_df(gdf_carroye_city_kept, crs_angles=crs_angles, crs_meters=crs_meters)

In [26]:
gdf_city_200_with_dist

,id,estimated,population,geometry,density_km2,ratio_appart,niveau_de_vie_moyen,centroid,dist_0,dist_1,...,dist_423,dist_424,dist_425,dist_426,dist_427,dist_428,dist_429,dist_430,longitude,latitude
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,0.150000,29316.704762,POINT (-2.3361 47.26818),0.000000,0.200250,...,5.534841,5.436998,4.884685,4.848552,4.820473,5.018388,5.790279,5.750147,-2.336104,47.268182
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,0.000000,23425.138462,POINT (-2.33349 47.26848),0.200250,0.000000,...,5.638107,5.534847,4.928702,4.884691,4.848558,5.045333,5.882190,5.801296,-2.333492,47.268481
2,CRS3035RES200mN2761400E3392000,1,2.0,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,...",50.086451,0.000000,28876.200000,POINT (-2.33088 47.26878),0.400500,0.200250,...,5.746497,5.638112,4.980388,4.928708,4.884698,5.080036,5.979397,5.858848,-2.330880,47.268780
3,CRS3035RES200mN2761600E3391200,0,93.5,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009...",2341.540973,0.542373,24798.703743,POINT (-2.34175 47.26936),0.446536,0.632069,...,5.167523,5.078366,4.622676,4.601897,4.589769,4.788994,5.440463,5.471568,-2.341752,47.269356
4,CRS3035RES200mN2761600E3391400,0,72.0,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039...",1803.111518,0.055556,27235.497222,POINT (-2.33914 47.26965),0.281920,0.446536,...,5.262799,5.167529,4.651996,4.622682,4.601903,4.800585,5.523716,5.510797,-2.339140,47.269655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,CRS3035RES200mN2766200E3390800,1,2.0,"POLYGON ((-2.35786 47.30847, -2.35828 47.31024...",50.085491,0.000000,22899.050000,POINT (-2.35676 47.30951),4.848552,4.884691,...,2.002564,1.802307,0.200255,0.000000,0.200255,0.283277,1.844047,0.996273,-2.356762,47.309506
427,CRS3035RES200mN2766200E3391000,1,3.0,"POLYGON ((-2.35524 47.30877, -2.35567 47.31054...",75.128225,0.000000,27568.400000,POINT (-2.35415 47.30981),4.820473,4.848558,...,2.202820,2.002562,0.400511,0.200255,0.000000,0.199407,2.040023,1.127722,-2.354148,47.309805
428,CRS3035RES200mN2766400E3391000,1,2.0,"POLYGON ((-2.35567 47.31054, -2.35609 47.31231...",50.085440,0.000000,27568.400000,POINT (-2.35457 47.31158),5.018388,5.045333,...,2.212775,2.013413,0.448256,0.283277,0.199407,0.000000,2.011522,0.997467,-2.354574,47.311577
429,CRS3035RES200mN2766600E3389000,1,3.0,"POLYGON ((-2.38223 47.30932, -2.38266 47.31109...",75.128205,0.090909,18076.700000,POINT (-2.38114 47.31035),5.790279,5.882190,...,0.447117,0.398808,1.649099,1.844047,2.040023,2.011522,0.000000,1.267803,-2.381138,47.310352


In [28]:
# Clean columns
cols = ['id', 'geometry', 'population', 'density_km2', 'niveau_de_vie_moyen']
cols += [f'dist_{i}' for i in range(len(gdf_city_200_with_dist))]

gdf_city_with_dist = gdf_city_200_with_dist[cols]
gdf_city_with_dist.to_csv(export_file_df_adjacency, index=False)
print(f"Dataframe saved at {export_file_df_adjacency}")

np.save(export_file_np_adjacency, adjacency_matrix)
print(f"Adjacency matrix alone saved at {export_file_np_adjacency}")

Dataframe saved at ../data/processed/gdf_city_200_with_dist_44055.csv
Adjacency matrix alone saved at ../data/processed/adjacency_matrix_200_44055.npy


### Toutes les données

In [7]:
import branca


m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city_kept['population'].min()  # Minimum number of people
max_people = gdf_carroye_city_kept['population'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city_kept.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    num_people = row['population']  # Assuming this column exists
    # num_menages = row['Men']  # Assuming this column exists
    cond = row['estimated'] == 1

    # Create a popup displaying the number of people
    text = f"Number of people: {num_people}"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, num_people=num_people: {
                'fillColor': colormap(num_people),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m